In [1]:
import time, os, threading, cv2
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from datetime import datetime

In [2]:
def scrape(path, headless=False , id=0):


    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--hide-scrollbars")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(path)
    driver.maximize_window()
    
    try:
        cookie = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler")))
        cookie.click()
    except:
        print("No cookie")
    
    try:
        login = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/a[@id="loginBtn"]')))
        login.click()

        user_name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "Username")))
        user_name.send_keys("sultanmirza0501@gmail.com")

        password = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "Password")))
        password.send_keys("Muhssan7865")

        login_tab = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Login']")
        driver.execute_script("arguments[0].scrollIntoView();", login_tab)
        login_tab.click()
        driver.back()
    except Exception:
        print("No login required or login failed.")
        
    
    # Navigate to Live Auctions
    try:
        # auc = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//a[text()="Auctions"])[1]')))
        # auc.click()

        # lives = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/a[text()="Open LIVE"]')))
        # lives[0].click()

        # WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/a[@class="button full-width mb-0"]'))).click()
        driver.get(f"https://www.astonbarclay.net/FMLiveToken/?saleid={id}&hall=")

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, './/a[@class="button full-width mb-0"]'))
        ).click()
    except Exception:
        print("Live auction not found.")
    
    time.sleep(5)
    return driver

# def record_website_screen(driver, output_file="live_rec.avi", fps=8):
#     try:
#         img = driver.get_screenshot_as_png()
#         nparr = np.frombuffer(img, np.uint8)
#         frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#         height, width, _ = frame.shape
#         fourcc = cv2.VideoWriter_fourcc(*'XVID')
#         out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

#         frame_count = 0
#         while True:
#             img = driver.get_screenshot_as_png()
#             nparr = np.frombuffer(img, np.uint8)
#             frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#             frame_count += 1
#             out.write(frame)
#             time.sleep(1/fps)
#     except Exception as e:
#         print(f"Error in screen recording: {e}")
#     finally:
#         out.release()
#         print(f"Recording saved, captured {frame_count} frames.")

# def start_recording(driver):
#     thread = threading.Thread(target=record_website_screen, args=(driver,))
#     thread.daemon = True
#     thread.start()
def save_full_screenshot(driver, prefix="bidding_status"):
    try:
        os.makedirs("screenshots", exist_ok=True)
        filename = f"screenshots/{prefix}.png"
        driver.save_screenshot(filename)
        print(f"📸 Screenshot saved: {filename}")
    except Exception as e:
        print(f"❌ Screenshot failed: {e}")

if __name__ == "__main__":
    path = "https://www.astonbarclay.net/auctions"
    driver = scrape(path, headless=False,id=17522)
    print("Starting screen recording...")
    # start_recording(driver)
    
    print("Starting data extraction...")
    results = []
    try:
        registration = ""
        while True:
            if not driver.window_handles:
                print("Browser window closed. Stopping script.")
                break
            details = {}
            try:
                try:
                    Reg = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="currentLot-title"])[2]')))
                    curr_reg = Reg.text.strip()
                    
                    if curr_reg != registration:
                        registration = curr_reg
                        details['Reg'] = curr_reg.split(" - ")[1]
                        
                        
                    try:
                        data_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="bidding-history-messages "]')))
                        prices_elements = WebDriverWait(data_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="d-flex flex-row align-items-center history-text wordwrap"]')))
                        price = [p.text.strip() for p in prices_elements]
                        details['Bidding History'] = price
                        for text in price:
                            if "SOLD" in text.upper():
                                save_full_screenshot(driver, prefix=registration)
                                break
                    except Exception as e:
                        print(f"Error getting prices")
                    
                except Exception as e:
                    print(f"Error getting Reg:")
                except StaleElementReferenceException:
                    print("Stale element, retrying...")
                    continue
            except:
                pass
            time.sleep(2)

            if details:
                # Write to CSV incrementally
                results.append(details)
                df = pd.DataFrame(results, columns=["Reg", "Bidding History"])
                # header = not os.path.exists("live_data.csv")
                df.to_csv("live_data.csv", mode='a', header = not os.path.exists("live_data.csv"), index=False)
            
    except KeyboardInterrupt:
        print("Stopped by user.")
    finally:
        driver.quit()
        print("Browser closed.")


Live auction not found.
Starting screen recording...
Starting data extraction...
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved: screenshots/335 - YA21WOC.png
📸 Screenshot saved:

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=141.0.7390.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x12dfe43+66515]
	GetHandleVerifier [0x0x12dfe84+66580]
	(No symbol) [0x0x10cdc48]
	(No symbol) [0x0x10bb9a4]
	(No symbol) [0x0x10bc937]
	(No symbol) [0x0x10ce6a3]
	(No symbol) [0x0x10abcf5]
	(No symbol) [0x0x114a708]
	(No symbol) [0x0x113b193]
	(No symbol) [0x0x110c2ca]
	(No symbol) [0x0x110d154]
	GetHandleVerifier [0x0x1537353+2521315]
	GetHandleVerifier [0x0x15322d3+2500707]
	GetHandleVerifier [0x0x1307c94+229924]
	GetHandleVerifier [0x0x12f81f8+165768]
	GetHandleVerifier [0x0x12fecad+193085]
	GetHandleVerifier [0x0x12e8158+100072]
	GetHandleVerifier [0x0x12e82f0+100480]
	GetHandleVerifier [0x0x12d25aa+11066]
	B

In [ ]:
df = pd.read_csv("live_data.csv")

df2 = df.drop_duplicates(subset=['Reg'], keep='first')

finalized_df = df2.to_csv("live_data_CLEANED.csv", index=False)
# delete the original df
os.remove("live_data.csv")

In [ ]:
df1 = pd.read_csv("live_data_CLEANED.csv")

df2 = pd.read_csv("barclay_renamed.csv")

merged_df = df2.merge(df1[['Reg',"Bidding History"]], on='Reg', how='left')


output_path = "barclay_live_data_merged_final.csv"
merged_df.to_csv(output_path, index=False)

df = pd.read_csv("barclay_live_data_merged_final.csv")

df.to_csv("Final_Barclay.csv", index=False)


In [ ]:
df = pd.read_csv("Final_Barclay.csv")
df

,Title,Derivative,Images,Auction Availability,Lot,Start Time,Start Date,Center,Auction ID,D.O.R,...,Tyre OSR,Tyre NSR,Spare Tyre,Equipment,Damaged_images,Damage_details,Year,Last Service,Last Service Mileage,Bidding History
0,AW19AWV - FORD FOCUS HATCHBACK 1.0 EcoBoost 12...,EcoBoost 125 ST-Line 5dr Petrol WHITE,https://abimg003.imgix.net/AW19AWV/2025-07-07/...,Available at auction,2,14:00,24/07/2025,Donington Park,16497,31/05/2019,...,"['', '5']","['', '6']","['', '']","['Alloys', 'Parking sensors', 'Heated windscre...",https://abimg003.imgix.net/AW19AWV/2025-07-07/...,"[{'Location': 'Door nsr'}, {'Fault': 'Poor Pre...",2019,02/06/24,na,NaN
1,GF21VLE - FORD FOCUS HATCHBACK 1.0 EcoBoost Hy...,EcoBoost Hybrid mHEV 155 Active X Edition 5dr ...,https://abimg003.imgix.net/GF21VLE/2025-07-24/...,Available at auction,3,14:00,24/07/2025,Donington Park,16497,30/07/2021,...,"['', '7']","['', '7']","['', '']",['No damage report data'],https://catalogue.astonbarclay.net/img/no-imag...,"[{'Location': 'Door nsr'}, {'Fault': 'Dented 2...",2021,02/07/21,na,NaN
2,RA24FGX - KIA SPORTAGE ESTATE 1.6T GDi 207 HEV...,GDi 207 HEV 3 5dr Auto Petrol/Electric Hybrid ...,https://abimg003.imgix.net/RA24FGX/2025-07-23/...,Available at auction,10,14:00,24/07/2025,Donington Park,16497,15/07/2024,...,"['', '5']","['', '5']","['', '']","['Air con', 'Sat nav']",https://abimg003.imgix.net/RA24FGX/2025-07-23/...,"[{'Location': 'Wing osf'}, {'Fault': 'Scratche...",2024,02/06/25,8k,NaN
3,RV74EZZ - KIA SPORTAGE ESTATE 1.6T GDi 207 HEV...,GDi 207 HEV 3 5dr Auto Petrol/Electric Hybrid ...,https://abimg003.imgix.net/RV74EZZ/2025-07-10/...,Available at auction,12,14:00,24/07/2025,Donington Park,16497,30/09/2024,...,"['', '4']","['', '4']","['', '']","['Alloys', 'Parking sensors', 'Heated seats', ...",https://abimg003.imgix.net/RV74EZZ/2025-07-10/...,"[{'Location': 'Door nsr'}, {'Fault': 'Dented B...",2024,02/06/25,9k,NaN
4,VK74RZR - VOLKSWAGEN TIGUAN ESTATE 1.5 TSI eHy...,TSI eHybrid R-Line 5dr DSG Hybrid Electric WHITE,https://abimg003.imgix.net/VK74RZR/2025-07-22/...,Available at auction,20,14:00,24/07/2025,Donington Park,16497,31/10/2024,...,"['', '7']","['', '5']","['', '']","['Air con', 'Sat nav']",https://abimg003.imgix.net/VK74RZR/2025-07-22/...,"[{'Location': 'Bumper Front'}, {'Fault': 'Chip...",2024,na,na,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,NL20LLR - NISSAN JUKE HATCHBACK 1.0 DiG-T Acen...,DiG-T Acenta 5dr Petrol WHITE,https://abimg003.imgix.net/NL20LLR/2025-07-15/...,Available at auction,277,14:00,24/07/2025,Donington Park,16497,15/03/2020,...,"['', '4']","['', '4']","['', '']","['Alloys', 'Parking sensors', 'AUX/USB', 'Radi...",https://abimg003.imgix.net/NL20LLR/2025-07-15/...,"[{'Location': 'Bumper Front'}, {'Fault': 'Chip...",2020,02/02/25,na,NaN
128,YS20MYY - FORD KUGA DIESEL ESTATE 1.5 EcoBlue ...,1.5 EcoBlue Zetec 5dr Diesel BLACK,https://abimg003.imgix.net/YS20MYY/2025-06-18/...,Available at auction,278,14:00,24/07/2025,Donington Park,16497,28/07/2020,...,"['', '5']","['', '5']","['', '']","['Alloys', 'AUX/USB', 'Cruise control', 'Air c...",https://abimg003.imgix.net/YS20MYY/2025-06-18/...,"[{'Location': 'Door nsr'}, {'Fault': 'Dented W...",2020,na,na,NaN
129,YH73JYN - ISUZU D-MAX DIESEL 1.9 DL40 Double C...,DL40 Double Cab 4x4 Diesel SILVER,https://abimg003.imgix.net/YH73JYN/2025-07-21/...,Available at auction,279,14:00,24/07/2025,Donington Park,16497,04/09/2023,...,"['', '7']","['', '8']","['', '']","['Towbar', 'Alloys', 'Parking sensors', 'Heate...",https://abimg003.imgix.net/YH73JYN/2025-07-21/...,"[{'Location': 'C Post ns'}, {'Fault': 'Scratch...",2023,02/MD,na,NaN
130,YD22KUF - TOYOTA HILUX DIESEL Invincible X D/C...,X D/Cab Pick Up 2.8 D-4D Diesel GREY,https://abimg003.imgix.net/YD22KUF/2025-07-11/...,Available at auction,280,14:00,24/07/2025,Donington Park,16497,28/03/2022,...,"['', '4']","['', '4']","['', '']","['Alloys', 'Parking sensors', 'Heated seats', ...",https://abimg003.imgix